We use the mnist dataset from keras.datasets library to load the MNIST dataset. The load_data() function returns two tuples of the training and test sets, respectively.

In [7]:
#load the MNIST data set
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

We use the train_test_split() function from sklearn.model_selection to split the training data into a training set and a development (validation) set. Here, we set the test_size parameter to 0.2, which means that 20% of the training data will be used as the development set. We also set the random_state parameter to a fixed value of 42 to ensure reproducibility of the results.

In [14]:
# Split the training data into a training and development set
from sklearn.model_selection import train_test_split

x_train, x_dev, y_train, y_dev = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

We use the RandomForestClassifier from sklearn.ensemble to create a random forest classifier. Here, we set the number of decision trees to 100 and the maximum depth of each tree to 20. The max_depth parameter controls the maximum depth of each decision tree in the forest, and it is the parameter we choose to tune. We use the fit() function to train the classifier on the training set.

In [13]:
# Train a random forest classifier with 100 decision trees and tune the `max_depth` parameter
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

rf = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42) # max_depth parameter is tuned here
rf.fit(x_train.reshape(len(x_train), -1), y_train)

RandomForestClassifier(max_depth=20, random_state=42)

We evaluate the performance of the model on the development set to choose the best value of max_depth. First, we use the trained classifier to make predictions on the development set using the predict() function. Then, we compute the accuracy of the predictions using the accuracy_score() function from sklearn.metrics. We set best_accuracy to the initial accuracy score, and best_max_depth to the initial max_depth value of 20. We then loop over a range of max_depth values from 5 to 25 with a step size of 5, and for each value, we train a new random forest classifier on the training set with the given max_depth value. We then use the trained classifier to make predictions on the development set and compute the accuracy of the predictions. If the accuracy is better than the previous best accuracy.

In [11]:
# Evaluate the performance of the model on the development set to choose the best `max_depth` value
y_dev_pred = rf.predict(x_dev.reshape(len(x_dev), -1))
best_accuracy = accuracy_score(y_dev, y_dev_pred)
best_max_depth = 20
for max_depth in range(5, 26, 5): # test different values of max_depth
    rf = RandomForestClassifier(n_estimators=100, max_depth=max_depth, random_state=42)
    rf.fit(x_train.reshape(len(x_train), -1), y_train)
    y_dev_pred = rf.predict(x_dev.reshape(len(x_dev), -1))
    accuracy = accuracy_score(y_dev, y_dev_pred)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_max_depth = max_depth

In [12]:
# Test the model with the best `max_depth` value on the test set and print out the confusion matrix
rf = RandomForestClassifier(n_estimators=100, max_depth=best_max_depth, random_state=42)
rf.fit(x_train.reshape(len(x_train), -1), y_train)
y_test_pred = rf.predict(x_test.reshape(len(x_test), -1))
print(confusion_matrix(y_test, y_test_pred))

# Print a report on the performance of the model, including accuracy, precision, recall, and f1-score for each class
print(classification_report(y_test, y_test_pred))

[[ 968    0    0    0    0    5    4    1    2    0]
 [   0 1124    2    2    1    1    3    0    1    1]
 [   6    0  996    7    4    0    3    9    7    0]
 [   0    0   11  974    0    5    0    9    8    3]
 [   1    0    1    0  956    0    4    0    3   17]
 [   4    2    2   13    3  855    6    2    4    1]
 [   9    3    0    0    3    6  936    0    1    0]
 [   1    4   23    3    2    0    0  984    1   10]
 [   6    0    5    8    4    6    5    3  924   13]
 [   6    7    1   10   12    2    1    3    7  960]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       980
           1       0.99      0.99      0.99      1135
           2       0.96      0.97      0.96      1032
           3       0.96      0.96      0.96      1010
           4       0.97      0.97      0.97       982
           5       0.97      0.96      0.97       892
           6       0.97      0.98      0.97       958
           7       0.97      0.96   

The model appears to perform very well, with an accuracy of 0.97. The precision, recall, and F1-score are high for most classes, indicating that the model is able to correctly identify the majority of the samples in the test set.